In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("/content/Churn.csv")

In [ ]:
df.Tenure.unique()

In [ ]:
df.head(5)

EDA-Exploratory Data Analysis


In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.drop(["RowNumber","CustomerId","Surname"],axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
cols_features=list(set(df.columns))

In [ ]:
cat_feat=['IsActiveMember',
 'NumOfProducts',
 'Geography',
 'Gender',
 'Exited',
 'HasCrCard','Tenure']

num_feat=list(set(cols_features)-set(cat_feat))

In [ ]:
df.head(3)

In [ ]:
cat_feat

In [ ]:
num_feat

In [ ]:
for i in cat_feat:
  print(i)
  print(df[i].value_counts())
  print(df[i].value_counts(normalize=True)*100)
  print(df[i].value_counts().plot.bar())
  plt.show()
  print("*"*50)

In [ ]:

for i in num_feat:
  print(i)
  sns.boxplot(df[i])
  plt.title(f"BOXPLOT Of: {i}")
  plt.show()
  plt.hist(df[i])
  plt.title(f"distribution: {i}")
  plt.show()
  print("*"*50)

In [ ]:
# bivariate analysis
for i in cat_feat:
  print(i)
  sns.boxplot(x='Exited',y=i,data=df)
  plt.show()
  print("*"*50)

In [ ]:
sns.heatmap(df[num_feat].corr(),annot= True, cmap ='viridis')
plt.show()

In [ ]:
sns.pairplot(df[num_feat +["Exited"]],hue='Exited')

In [ ]:
# Bivarte analysis of categorical datapoints
for feat in cat_feat:
  # df.groupby(feat)['Exited'].mean()
  print(feat)
  df.groupby(feat)['Exited'].mean().plot.bar()
  plt.show()

# now build the base line model


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def base_model_0(row):
  return 0

def base_model(row):
  if row['Tenure']<3 and row['IsActiveMember']==0:
    return 1 # Churn
  else:
    return 0 # Not Churn

def base_model_2(row):
  if row['NumOfProducts']>2 :
    return 1
  else:
    return 0


def base_model_3(row):
  if (row['IsActiveMember'] == 0 and (row['Age'] > 40 or row['Geography'] == 'Germany')) or (row['NumOfProducts'] > 2):
    return 1 # Churn
  else:
    return 0 # Not Churn

def base_model_4(row):
  if row['Age']<18:
    return 1
  else:
    return 0

df['base_prediction']=df.apply(base_model_3,axis=1)

df['base_prediction_0']=df.apply(base_model_0,axis=1)
print(classification_report(df['Exited'],df['base_prediction_0']))

df['base_prediction']=df.apply(base_model,axis=1)
print(classification_report(df['Exited'],df['base_prediction']))

df['base_prediction_2']=df.apply(base_model_2, axis=1)
print(classification_report(df['Exited'],df['base_prediction_2']))

df['base_prediction_3']=df.apply(base_model_3, axis=1)
print(classification_report(df['Exited'],df['base_prediction_3']))

df['base_prediction_4']=df.apply(base_model_4, axis=1)
print(classification_report(df['Exited'],df['base_prediction_4']))



#machine learning model


In [ ]:
df1=cat_feat+num_feat
df1

In [ ]:
df['Geography'].unique()

In [ ]:
df['Geography'].isna().sum()

In [ ]:
df['Gender']=df['Gender'].map({'Female':1, 'Male':0}).astype('int')
df['Geography']=df['Geography'].map({'France':0, 'Spain':1, 'Germany':2}).astype('int')

In [ ]:
df[['Geography', 'Gender']].dtypes

In [ ]:
df.head(2)

In [ ]:
#cat_feat

In [ ]:
x = df.drop(['Exited'],axis=1)

y = df["Exited"]

In [ ]:
x.head(3)

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
print(x_train.shape,x_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
y_pred_lr=log_reg.predict(x_test)
print(classification_report(y_test,y_pred_lr))

In [ ]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=0)
dt.fit(x_train,y_train)

In [ ]:
y_pred_dt=dt.predict(x_test)
print(classification_report(y_test,y_pred_dt))

DL- Deep Learning Model


In [ ]:
#!pip install keras

In [ ]:
# !pip install pydot
# !pip install tensorflow

In [ ]:
# Imports
from keras.models import Sequential
from keras.layers import Dense, Input
import keras as keras
from keras import optimizers
from keras import metrics


In [ ]:
# Standardization of data
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(x_train)
X_test=sc.transform(x_test)

In [ ]:
classifier = Sequential()
classifier.add(Dense(10,activation='relu',input_dim=X_train.shape[1])) # Hidden Layer-1
classifier.add(Dense(7,activation='relu'))
# classifier.add(Dense(32,activation='relu'))
# classifier.add(Dense(32,activation='relu'))
classifier.add(Dense(1,activation='sigmoid')) # Output Layer
classifier.summary()

In [ ]:
#Compile
#Compile
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#training
history=classifier.fit(X_train,y_train,batch_size=32,epochs=10, validation_data=(X_test,y_test))

In [ ]:
keras.utils.plot_model(classifier,"my_first_model.png", show_shapes= True, show_layer_names= True)

In [ ]:
#List all data in history
print(history.history.keys())

#Plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


#Plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.layers import Input

In [ ]:
modelALT=Sequential([
    Dense(5, activation="relu", name="layer1",input_dim=x_train.shape[1]),
    Dense(1,activation="sigmoid", name= "layer2")
])
modelALT.summary()

modelALT.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
modelALT.fit(x_train,y_train, epochs=10, validation_data=(x_test,y_test))

In [ ]:
modelALT = Sequential([
    Input(shape=(15,)),          # ← matches x_train features
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

modelALT.compile(
    optimizer='SGD',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

modelALT.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [ ]:
#Predictions
y_pred_dl=classifier.predict(X_test)

In [ ]:
#ROC-AUC Curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred_dl)
roc_auc = auc(fpr, tpr)
print(roc_auc)

# Plot
plt.figure()
plt.plot(fpr,tpr, label=f"AUC = {roc_auc}")
plt.plot([0,1],[0,1],"r--", label="Random Guess")
plt.show()

In [ ]:

y_pred_dl1=[]
for i in y_pred_dl:
  if i>0.6:
    y_pred_dl1.append(1)
  else:
    y_pred_dl1.append(0)

print(classification_report(y_test,y_pred_dl1))

Keras Functional API
https://keras.io/api/models/


In [ ]:
input = keras.Input(shape=(x_train.shape[1],))
hidn=Dense(5,activation='relu')(input)
output=Dense(1,activation='sigmoid')(hidn)
model=keras.Model(inputs=input, outputs=output, name="churn_model")
model.summary()

In [ ]:
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10, validation_data=(x_test, y_test))

In [ ]:
# prompt: Create a ANN network to solve this problem and achieve better accuracy than decision tree used here

# Scale the numerical features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Build the ANN model
model = Sequential()
model.add(Dense(5, activation='relu', input_dim=15))
# model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10,  validation_data=(x_test, y_test))

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: {}'.format(accuracy))


In [ ]:
#Predictions
y_pred_dl=model.predict(x_test)

#ROC-AUC Curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred_dl)
roc_auc = auc(fpr, tpr)
print(roc_auc)

# Plot
plt.figure()
plt.plot(fpr,tpr, label=f"AUC = {roc_auc}")
plt.plot([0,1],[0,1],"r--", label="Random Guess")
plt.show()


y_pred_dl1=[]
for i in y_pred_dl:
  if i>0.6:
    y_pred_dl1.append(1)
  else:
    y_pred_dl1.append(0)

print(classification_report(y_test,y_pred_dl1))

In [ ]:
x.columns

In [ ]:
# Create a ANN network to solve this problem and achieve better accuracy than decision tree used here

# Scale the numerical features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Build the ANN model
# Build the ANN model
model = Sequential()

model.add(Dense(64, activation='relu', input_dim=15))
#Dropout
from keras import layers

model.add(layers.Dropout(0.5))

model.add(Dense(32, activation='relu'))
#Dropout
model.add(layers.Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Customize your optimizer
from keras.optimizers import Adam
custom_optimizer = Adam(learning_rate=0.9)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Add earlyStopping Callback
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Train the model
history=model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test),callbacks=[early_stopping])

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: {}'.format(accuracy))

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Create a ANN network to solve this problem and achieve better accuracy than decision tree used here

# Build the ANN model
model = Sequential()

# Correct input_dim to match the number of features in X_train (which is 15)
model.add(Dense(64, activation='relu', input_dim=15))
#Dropout
from keras import layers

model.add(layers.Dropout(0.5))

model.add(Dense(32, activation='relu'))
#Dropout
model.add(layers.Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Customize your optimizer
from keras.optimizers import Adam
custom_optimizer = Adam(learning_rate=0.9)



# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Add earlyStopping Callback
from keras.callbacks import EarlyStopping, LearningRateScheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
lr_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** epoch)

# Train the model
history=model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test),callbacks=[early_stopping,lr_scheduler])

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: {}'.format(accuracy))

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
epoch=10
0.001 * 0.9 ** epoch

In [ ]:
# track LR
plt.plot(history.history['learning_rate'])
plt.title('Learning Rate')
plt.ylabel('learning rate')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Create a ANN network to solve this problem and achieve better accuracy than decision tree used here

# Scale the numerical features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Build the ANN model
# Build the ANN model
model = Sequential()

model.add(Dense(64, activation='relu', input_dim=15))
#Dropout
model.add(layers.Dropout(0.5))

model.add(Dense(32, activation='relu'))
#Dropout
model.add(layers.Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Customize your optimizer
from keras.optimizers import Adam
custom_optimizer = Adam(learning_rate=0.9)



# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Add earlyStopping Callback
from keras.callbacks import EarlyStopping, LearningRateScheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
lr_scheduler = LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** epoch)
#Tensorboard
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='/content/sample_data/model_logs', histogram_freq=0, write_graph=True, write_images=True)

# Train the model
history=model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test),callbacks=[early_stopping,lr_scheduler,tensorboard])

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: {}'.format(accuracy))

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
